In [1]:
#hide
!pip install -Uqq fastbook
import fastbook
fastbook.setup_book()

     |████████████████████████████████| 720 kB 4.0 MB/s 
     |████████████████████████████████| 46 kB 4.9 MB/s 
     |████████████████████████████████| 1.2 MB 36.5 MB/s 
     |████████████████████████████████| 186 kB 40.8 MB/s 
     |████████████████████████████████| 56 kB 5.1 MB/s 
     |████████████████████████████████| 51 kB 390 kB/s 
Mounted at /content/gdrive


In [2]:
#hide
from fastai.vision.all import *
from fastbook import *

matplotlib.rc('image', cmap='Greys')

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
path = untar_data(URLs.MNIST)

In [5]:
Path.BASE_PATH = path

In [6]:
path.ls()

(#2) [Path('testing'),Path('training')]

In [7]:
(path/'training'/'0').ls()

(#5923) [Path('training/0/25278.png'),Path('training/0/27395.png'),Path('training/0/30790.png'),Path('training/0/55355.png'),Path('training/0/50463.png'),Path('training/0/13013.png'),Path('training/0/28855.png'),Path('training/0/57827.png'),Path('training/0/30293.png'),Path('training/0/53077.png')...]

In [8]:
def get_paths(path, folder='training'):
  paths = []
  for i in range(10):
    p = (path/folder/str(i)).ls()
    paths.append(p)
  return paths

In [9]:
paths = get_paths(path)

In [10]:
def get_tensors(paths):
  list_stacked_tensors = []
  for p in paths:
    p_tensors = [tensor(Image.open(o)) for o in p]
    p_stacked = torch.stack(p_tensors).float()/255
    list_stacked_tensors.append(p_stacked)
  return list_stacked_tensors

In [11]:
stacked_tensors = get_tensors(paths)

In [12]:
stacked_tensors[0].shape[0]

5923

In [13]:
train_x = torch.cat(stacked_tensors).view(-1, 28*28)
train_x.shape

torch.Size([60000, 784])

In [14]:
ls = [0]*10
ls[0] = 1
train_y = tensor(ls).repeat(stacked_tensors[0].shape[0], 1)
train_y.shape

torch.Size([5923, 10])

In [15]:
train_y[:5]

tensor([[1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 0, 0, 0, 0, 0, 0, 0, 0, 0]])

In [16]:
def get_y(stacked_tensors):
  train_y = []
  for i in range(len(stacked_tensors)):
    ls = [0]*10
    ls[i]=1
    y = tensor(ls).repeat(stacked_tensors[i].shape[0], 1)
    train_y.append(y)
  return torch.cat(train_y).view(-1, 10)


In [17]:
train_y = get_y(stacked_tensors)
train_y.shape

torch.Size([60000, 10])

In [18]:
train_y[5000]

tensor([1, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [19]:
train_x.shape

torch.Size([60000, 784])

In [20]:
train_y[5923-1:5923+20]

tensor([[1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 1, 0, 0, 0, 0, 0, 0, 0, 0]])

In [94]:
linear_model = nn.Sequential(
    nn.Linear(28*28, 16),
    nn.ReLU(),
    nn.Linear(16, 16),
    nn.ReLU(),
    nn.Linear(16, 10)
)

In [22]:
linear_model(train_x[:10])


tensor([[-1.8845e-01, -8.6392e-02,  5.9547e-02, -2.1734e-01,  1.5739e-01,  3.6190e-03,  1.9846e-01, -8.9618e-02, -9.3726e-02,  3.8116e-02],
        [-2.0177e-01, -8.5064e-02,  6.3393e-02, -2.1304e-01,  1.6602e-01,  4.8716e-03,  1.9347e-01, -9.1650e-02, -9.9455e-02,  3.7658e-02],
        [-1.5413e-01, -8.1747e-02,  5.3284e-02, -2.2706e-01,  1.1881e-01, -1.0763e-02,  1.7615e-01, -9.0531e-02, -6.8429e-02,  3.7012e-02],
        [-1.9397e-01, -7.2384e-02,  4.9884e-02, -2.2548e-01,  1.6681e-01,  2.7379e-02,  2.1876e-01, -1.1454e-01, -1.1000e-01,  2.6855e-02],
        [-1.9085e-01, -7.2622e-02,  4.2335e-02, -2.3657e-01,  1.5700e-01,  2.2409e-02,  2.1189e-01, -1.3037e-01, -9.7921e-02,  1.6482e-02],
        [-2.2025e-01, -7.0358e-02,  5.1810e-02, -2.7665e-01,  1.1444e-01,  1.2312e-01,  1.8849e-01, -1.8214e-01, -9.5172e-02,  8.4748e-03],
        [-2.2228e-01, -5.7639e-02,  4.1392e-02, -2.6327e-01,  1.4168e-01,  9.5045e-02,  2.0883e-01, -1.8562e-01, -1.0421e-01, -3.0033e-03],
        [-1.8808e-01

In [23]:
def mnist_loss(preds, targets):
  return ((preds-targets)**2).sqrt().mean()

In [24]:
preds = linear_model(train_x[:5])

In [25]:
loss = mnist_loss(preds, train_y[:5])
loss

tensor(0.2138, grad_fn=<MeanBackward0>)

In [26]:
dset = list(zip(train_x, train_y))

In [27]:
valid_paths = get_paths(path, 'testing')

In [28]:
stacked_valid = get_tensors(valid_paths)

In [29]:
len(stacked_valid)

10

In [30]:
valid_x = torch.cat(stacked_valid).view(-1, 28*28)
valid_x.shape

torch.Size([10000, 784])

In [31]:
valid_y = get_y(stacked_valid)
valid_y.shape

torch.Size([10000, 10])

In [32]:
valid_set = list(zip(valid_x, valid_y))

In [33]:
valid_set[1]

(tensor([0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0471,
         0.7529, 0.4627, 0.0706, 0.0510, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0

In [34]:
dl = DataLoader(dset, batch_size=256)
valid_dl = DataLoader(valid_set, batch_size=256)

In [35]:
lr = 1e-5
dls = DataLoaders(dl, valid_dl)

In [36]:
opt = SGD(linear_model.parameters(), lr)

In [84]:
def batch_accuracy(xb, yb):
  preds = xb.sigmoid()
  corrects = (xb.argmax(1) == yb.argmax(1))
  return corrects.float().mean()


In [85]:
l = linear_model(train_x[:10]).sigmoid()
l

tensor([[0.4532, 0.4785, 0.5148, 0.4459, 0.5392, 0.5008, 0.5493, 0.4778, 0.4767, 0.5095],
        [0.4499, 0.4789, 0.5158, 0.4470, 0.5413, 0.5012, 0.5481, 0.4773, 0.4752, 0.5094],
        [0.4617, 0.4797, 0.5132, 0.4435, 0.5295, 0.4972, 0.5438, 0.4776, 0.4830, 0.5092],
        [0.4518, 0.4820, 0.5124, 0.4439, 0.5415, 0.5068, 0.5543, 0.4716, 0.4726, 0.5066],
        [0.4526, 0.4819, 0.5105, 0.4412, 0.5391, 0.5056, 0.5527, 0.4676, 0.4756, 0.5040],
        [0.4453, 0.4825, 0.5128, 0.4313, 0.5284, 0.5306, 0.5469, 0.4548, 0.4764, 0.5020],
        [0.4448, 0.4857, 0.5102, 0.4346, 0.5352, 0.5237, 0.5519, 0.4539, 0.4741, 0.4991],
        [0.4532, 0.4799, 0.5132, 0.4455, 0.5401, 0.5026, 0.5523, 0.4750, 0.4759, 0.5086],
        [0.4622, 0.4787, 0.5163, 0.4495, 0.5309, 0.5000, 0.5491, 0.4851, 0.4811, 0.5162],
        [0.4522, 0.4822, 0.5091, 0.4349, 0.5344, 0.5209, 0.5538, 0.4604, 0.4738, 0.5035]], grad_fn=<SigmoidBackward>)

In [83]:
l.max(1)

torch.return_types.max(values=tensor([0.5493, 0.5481, 0.5438, 0.5543, 0.5527, 0.5469, 0.5519, 0.5523, 0.5491, 0.5538], grad_fn=<MaxBackward0>), indices=tensor([6, 6, 6, 6, 6, 6, 6, 6, 6, 6]))

In [78]:
torch.argmax(linear_model(train_x[:10]), dim=0)

tensor([8, 6, 8, 8, 3, 5, 3, 8, 2, 8])

In [86]:
batch_accuracy(linear_model(train_x[:10]), train_y[:10])

tensor(0.)

In [76]:
t = torch.rand((4,4)).sigmoid().requires_grad_()
t

tensor([[0.6283, 0.6550, 0.6175, 0.7259],
        [0.6947, 0.6486, 0.5140, 0.5649],
        [0.6672, 0.5441, 0.6546, 0.6509],
        [0.5463, 0.6455, 0.6071, 0.7052]], requires_grad=True)

In [77]:
t.argmax(1)

tensor([3, 0, 0, 3])

In [40]:
preds = linear_model(train_x[:10])
preds

tensor([[-1.8845e-01, -8.6392e-02,  5.9547e-02, -2.1734e-01,  1.5739e-01,  3.6190e-03,  1.9846e-01, -8.9618e-02, -9.3726e-02,  3.8116e-02],
        [-2.0177e-01, -8.5064e-02,  6.3393e-02, -2.1304e-01,  1.6602e-01,  4.8716e-03,  1.9347e-01, -9.1650e-02, -9.9455e-02,  3.7658e-02],
        [-1.5413e-01, -8.1747e-02,  5.3284e-02, -2.2706e-01,  1.1881e-01, -1.0763e-02,  1.7615e-01, -9.0531e-02, -6.8429e-02,  3.7012e-02],
        [-1.9397e-01, -7.2384e-02,  4.9884e-02, -2.2548e-01,  1.6681e-01,  2.7379e-02,  2.1876e-01, -1.1454e-01, -1.1000e-01,  2.6855e-02],
        [-1.9085e-01, -7.2622e-02,  4.2335e-02, -2.3657e-01,  1.5700e-01,  2.2409e-02,  2.1189e-01, -1.3037e-01, -9.7921e-02,  1.6482e-02],
        [-2.2025e-01, -7.0358e-02,  5.1810e-02, -2.7665e-01,  1.1444e-01,  1.2312e-01,  1.8849e-01, -1.8214e-01, -9.5172e-02,  8.4748e-03],
        [-2.2228e-01, -5.7639e-02,  4.1392e-02, -2.6327e-01,  1.4168e-01,  9.5045e-02,  2.0883e-01, -1.8562e-01, -1.0421e-01, -3.0033e-03],
        [-1.8808e-01

In [41]:
preds.argmax(0) == train_y[:10].argmax(0)

tensor([False, False, False, False, False, False, False, False, False, False])

In [42]:
train_y[6000:6000+10].argmax(1)

tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [43]:
linear_model(train_x[:5]).argmax(0)

tensor([2, 3, 1, 1, 3, 3, 3, 0, 2, 0])

In [44]:
ls = [0]*10
ls[5] = 1
t = tensor(ls)
t

tensor([0, 0, 0, 0, 0, 1, 0, 0, 0, 0])

In [45]:
t.argmax()

tensor(5)

In [91]:
learn = Learner(dls, linear_model, loss_func=mnist_loss, opt_func=SGD, lr=lr)

In [95]:
learn.fit(100)

epoch,train_loss,valid_loss,time
0,nan,nan,00:00
1,nan,nan,00:00
2,nan,nan,00:00
3,nan,nan,00:01
4,nan,nan,00:01
5,nan,nan,00:01


KeyboardInterrupt: ignored